<h1><center> NASA Airathon - NO2 Track </center></h1>

### <center> Extract GFS data corresponding to test cases </center>

<div style="text-align: center"> 
    Dr. Sukanta Basu <br/> Associate Professor <br/> Delft University of Technology, The Netherlands <br/> Email: s.basu@tudelft.nl<br/> https://sites.google.com/view/sukantabasu/
</div>

#### Log

Last updated: 4th April, 2022

Midnight to Midnight:
LAX: 8Z     - 32Z (I consider forecast hours: 6:3:30)
DEL: 18:30Z - 42:30Z (I consider forecast hours: 18:3:42)
TAI: 16Z    - 40Z (I consider forecast hours: 15:3:39)
Note: when 12 am starts at LAX, it is 8 pm at Taipei. 

#### User instructions

Before running this notebook, please extract the GFS files in individual sub-directories. See README_airathon.md for details.  

#### Load packages

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import netCDF4

#### Directories

In [2]:
ROOT_DIR    = '../../'

#Please update this directory depending on the location of the extracted GFS data
GFS_DIR     = '../../data/airathon/raw/GFS/'

#Location of processed datasets
EXTDATA_DIR = ROOT_DIR + 'data/airathon/processed/'

#### Load combo data

In [3]:
df_OBS    = pd.read_csv(EXTDATA_DIR + 'test/STN/' + 'testOBS.csv') #Contains: datetime, ID, latitude, longitude, NO2
nSamples  = np.size(df_OBS['datetime'])

In [4]:
LONx = df_OBS['longitude']
LATx = df_OBS['latitude']

YR = (pd.to_datetime(df_OBS['datetime'])).dt.year
MO = (pd.to_datetime(df_OBS['datetime'])).dt.month
DY = (pd.to_datetime(df_OBS['datetime'])).dt.day
JD = (pd.to_datetime(df_OBS['datetime'])).dt.dayofyear
WD = (pd.to_datetime(df_OBS['datetime'])).dt.weekday
HR = 0 #Only consider 0 UTC cycle

#### Compute nearest point

In [5]:
def nearestGridFAST(LAT,LON,LATx,LONx):
    
    dLAT = np.abs(LAT - LATx)
    dLON = np.abs(LON - LONx)
    
    dTOT = dLAT + dLON #taxi-cab distance
    
    #Interesting function; https://stackoverflow.com/questions/3230067/numpy-minimum-in-row-column-format
    r_min, c_min = np.unravel_index(dTOT.argmin(), dTOT.shape)
    
    return r_min, c_min

#### Extract GFS

In [6]:
GFSdata = np.zeros((nSamples,13*9)) #13 variables (PBLH, M10, M100, alpha, sinX100, cosX100, 
                                    #beta, SHFX, VENT, RH, T2, T100-T2, Rig); 
                                    #24 h = 9 x 3-hourly samples spanning 0 h to 0 h

for n in range(nSamples):
    
    if (LONx[n] > 120) & (LONx[n] < 122):
        strCITY = 'TAI/'
        fcstHRini = 15
        fcstHRfin = 39+1
    elif (LONx[n] > 76) & (LONx[n] < 78):
        strCITY = 'DEL/'
        fcstHRini = 18
        fcstHRfin = 42+1
    elif (LONx[n] > -119) & (LONx[n] < -116):
        strCITY = 'LAX/'
        fcstHRini = 6
        fcstHRfin = 30+1        
    else:
        strCITY = 'XXX/'
    
    cnt = 0
    for fcstHR in np.arange(fcstHRini,fcstHRfin,3): #I use data from 3 UTC to 24 UTC
        
        strFILE = 'gfs.0p25.' + str(YR[n]) + str(MO[n]).zfill(2) + str(DY[n]).zfill(2) + str(HR).zfill(2) + '.f' + str(fcstHR).zfill(3) + '.grib2.nc'
        
        #-----------------
        strVAR = 'PBLH/'
        filePBLH = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataPBLH = netCDF4.Dataset(filePBLH,'r')
        
        LAT  = dataPBLH.variables['lat'][:]
        LON  = dataPBLH.variables['lon'][:]
        if (strCITY == 'LAX/'): 
            LON = LON - 360 #GFS outputs data as 240 E instead of -120 W
        LON2d, LAT2d = np.meshgrid(LON,LAT)
            
        PBLH         = np.squeeze(dataPBLH.variables['HPBL_L1'][:])  
        
        r_min, c_min = nearestGridFAST(LAT2d,LON2d,LATx[n],LONx[n])
        
        PBLHi = PBLH[r_min][c_min]
        GFSdata[n,cnt] = PBLHi
        cnt = cnt + 1
        #-----------------
        
        #-----------------
        strVAR    = 'U/'
        fileU     = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataU     = netCDF4.Dataset(fileU,'r')
        U         = np.squeeze(dataU.variables['U_GRD_L103'][:])     
        U100      = np.squeeze(U[0,:,:])
        U10       = np.squeeze(U[1,:,:])
        U100i     = U100[r_min][c_min]
        U10i      = U10[r_min][c_min]
        #-----------------
        
        #-----------------
        strVAR    = 'V/'
        fileV     = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataV     = netCDF4.Dataset(fileV,'r')
        V         = np.squeeze(dataV.variables['V_GRD_L103'][:])     
        V100      = np.squeeze(V[0,:,:])
        V10       = np.squeeze(V[1,:,:])
        V100i     = V100[r_min][c_min]
        V10i      = V10[r_min][c_min]
        #-----------------

        #-----------------
        M10i      = np.sqrt(U10i**2  + V10i**2)
        M100i     = np.sqrt(U100i**2 + V100i**2)
        alpha     = np.log(M100i/M10i)/np.log(10)
        
        RtoD      = 180/np.pi
        X10i      = np.arctan2(-U10i,-V10i)*RtoD
        if X10i < 0:
            X10i = X10i + 360
        X100i     = np.arctan2(-U100i,-V100i)*RtoD
        if X100i < 0:
            X100i = X100i + 360
        sinX100i  = np.sin(X100i/RtoD)
        cosX100i  = np.cos(X100i/RtoD)
        beta      = np.abs(X100i-X10i)
        if beta > 180:
            beta = 360 - beta
        
        GFSdata[n,cnt] = M10i
        cnt = cnt + 1
        
        GFSdata[n,cnt] = M100i
        cnt = cnt + 1
        
        GFSdata[n,cnt] = alpha 
        cnt = cnt + 1
        
        GFSdata[n,cnt] = sinX100i 
        cnt = cnt + 1

        GFSdata[n,cnt] = cosX100i 
        cnt = cnt + 1

        GFSdata[n,cnt] = beta 
        cnt = cnt + 1
        #-----------------

        #-----------------
        strVAR    = 'SHFX/'
        fileSHFX  = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataSHFX  = netCDF4.Dataset(fileSHFX,'r')
        SHFX      = np.squeeze(dataSHFX.variables['SHTFL_L1_Avg_1'][:])     
        SHFXi     = SHFX[r_min][c_min]

        GFSdata[n,cnt] = SHFXi
        cnt = cnt + 1        
        #-----------------

        #-----------------
        strVAR    = 'VENT/'
        fileVENT  = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataVENT  = netCDF4.Dataset(fileVENT,'r')
        VENT      = np.squeeze(dataVENT.variables['VRATE_L220'][:])     
        VENTi     = VENT[r_min][c_min]

        GFSdata[n,cnt] = VENTi
        cnt = cnt + 1        
        #-----------------

        #-----------------
        strVAR    = 'RH/'
        fileRH    = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataRH    = netCDF4.Dataset(fileRH,'r')
        RH        = np.squeeze(dataRH.variables['R_H_L103'][:])     
        RHi       = RH[r_min][c_min]

        GFSdata[n,cnt] = RHi
        cnt = cnt + 1        
        #-----------------

        #-----------------
        strVAR    = 'T2/'
        fileT2    = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataT2    = netCDF4.Dataset(fileT2,'r')
        T2        = np.squeeze(dataT2.variables['TMP_L103'][:])     
        T2i       = T2[r_min][c_min]

        GFSdata[n,cnt] = T2i
        cnt = cnt + 1        
        #-----------------

        #-----------------
        strVAR    = 'T100/'
        fileT100  = Path(GFS_DIR + strCITY + strVAR + strFILE)
        dataT100  = netCDF4.Dataset(fileT100,'r')
        T100      = np.squeeze(dataT100.variables['TMP_L103'][:])     
        T100i     = T100[r_min][c_min]

        GFSdata[n,cnt] = T100i - T2i
        cnt = cnt + 1        
        #-----------------

        #-----------------
        GFSdata[n,cnt] = (T100i - T2i)/(M100i - M10i)**2
        cnt = cnt + 1                
        #-----------------

        print((n,nSamples))

(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(0, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(1, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(2, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(3, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(4, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(5, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(6, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(7, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(8, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(9, 16350)
(10, 16350

(79, 16350)
(79, 16350)
(79, 16350)
(79, 16350)
(79, 16350)
(79, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(80, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(81, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(82, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(83, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(84, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(85, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(86, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(87, 16350)
(88, 16350)
(88, 16350)
(88, 16350)
(88, 16350)
(88, 16350)
(88,

(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(152, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(153, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(154, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(155, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(156, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(157, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(158, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(159, 16350)
(160, 16350)
(160, 16350)
(160, 16350)
(160, 16350)
(160, 16350)

(225, 16350)
(225, 16350)
(225, 16350)
(225, 16350)
(225, 16350)
(225, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(226, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(227, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(228, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(229, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(230, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(231, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(232, 16350)
(233, 16350)
(233, 16350)
(233, 16350)
(233, 16350)
(233, 16350)
(233, 16350)
(233, 16350)
(233, 16350)

(295, 16350)
(295, 16350)
(295, 16350)
(295, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(296, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(297, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(298, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(299, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(300, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(301, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(302, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(303, 16350)
(304, 16350)

...

(16048, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16049, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16050, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16051, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16052, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16053, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16054, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16055, 16350)
(16056, 16350)
(16056, 16350)
(16056, 16

(16112, 16350)
(16112, 16350)
(16112, 16350)
(16112, 16350)
(16112, 16350)
(16112, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16113, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16114, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16115, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16116, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16117, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16118, 16350)
(16119, 16350)
(16119, 16350)
(16119, 16350)
(16119, 16350)
(16119, 16350)
(16119, 16350)
(16119, 16

(16175, 16350)
(16175, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16176, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16177, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16178, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16179, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16180, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16181, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16182, 16350)
(16183, 16350)
(16183, 16

(16237, 16350)
(16237, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16238, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16239, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16240, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16241, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16242, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16243, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16244, 16350)
(16245, 16350)
(16245, 16

(16302, 16350)
(16302, 16350)
(16302, 16350)
(16302, 16350)
(16302, 16350)
(16302, 16350)
(16302, 16350)
(16302, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16303, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16304, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16305, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16306, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16307, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16308, 16350)
(16309, 16350)
(16309, 16350)
(16309, 16350)
(16309, 16350)
(16309, 16

In [7]:
df_new = pd.DataFrame(data=GFSdata)
df_new.columns = ['PBLH_0' ,'M10_0' ,'M100_0' ,'alpha_0', 'sinX100_0', 'cosX100_0', 
                  'beta_0', 'SHFX_0', 'VENT_0', 'RH_0', 'T2_0', 'dT_0', 'Rig_0',
                  'PBLH_3' ,'M10_3' ,'M100_3' ,'alpha_3', 'sinX100_3', 'cosX100_3', 
                  'beta_3', 'SHFX_3', 'VENT_3', 'RH_3', 'T2_3', 'dT_3', 'Rig_3',
                  'PBLH_6' ,'M10_6' ,'M100_6' ,'alpha_6', 'sinX100_6', 'cosX100_6', 
                  'beta_6', 'SHFX_6', 'VENT_6', 'RH_6', 'T2_6', 'dT_6', 'Rig_6',
                  'PBLH_9' ,'M10_9' ,'M100_9' ,'alpha_9', 'sinX100_9', 'cosX100_9', 
                  'beta_9', 'SHFX_9', 'VENT_9', 'RH_9', 'T2_9', 'dT_9', 'Rig_9',
                  'PBLH_12','M10_12','M100_12','alpha_12','sinX100_12','cosX100_12',
                  'beta_12','SHFX_12','VENT_12','RH_12','T2_12','dT_12','Rig_12',
                  'PBLH_15','M10_15','M100_15','alpha_15','sinX100_15','cosX100_15',
                  'beta_15','SHFX_15','VENT_15','RH_15','T2_15','dT_15','Rig_15',
                  'PBLH_18','M10_18','M100_18','alpha_18','sinX100_18','cosX100_18',
                  'beta_18','SHFX_18','VENT_18','RH_18','T2_18','dT_18', 'Rig_18',
                  'PBLH_21','M10_21','M100_21','alpha_21','sinX100_21','cosX100_21',
                  'beta_21','SHFX_21','VENT_21','RH_21','T2_21','dT_21', 'Rig_21',
                  'PBLH_24','M10_24','M100_24','alpha_24','sinX100_24','cosX100_24',
                  'beta_24','SHFX_24','VENT_24','RH_24','T2_24','dT_24', 'Rig_24']       
    
df_new.insert(0, 'WDAY', WD)
df_new.insert(1, 'sinJDAY', np.sin(2*np.pi*JD/365))
df_new.insert(2, 'cosJDAY', np.cos(2*np.pi*JD/365))

In [8]:
print(df_new)

       WDAY   sinJDAY   cosJDAY      PBLH_0     M10_0    M100_0   alpha_0  \
0         5 -0.924291 -0.381689   27.040661  0.870090  1.487965  0.233029   
1         5 -0.924291 -0.381689  152.400665  2.102432  2.373943  0.052748   
2         5 -0.924291 -0.381689  122.480659  1.325305  1.451132  0.039391   
3         5 -0.924291 -0.381689  103.120659  1.218181  1.089914 -0.048320   
4         5 -0.924291 -0.381689   93.120659  1.542322  1.933030  0.098064   
...     ...       ...       ...         ...       ...       ...       ...   
16345     1 -0.796183 -0.605056   11.255896  1.157034  1.002002 -0.062478   
16346     1 -0.796183 -0.605056  380.375885  2.246919  2.460102  0.039366   
16347     1 -0.796183 -0.605056  372.455902  2.207435  2.172388 -0.006951   
16348     1 -0.796183 -0.605056   78.455894  0.819009  1.989832  0.385528   
16349     1 -0.796183 -0.605056   10.615895  0.783059  0.647857 -0.082315   

       sinX100_0  cosX100_0     beta_0  ...  alpha_24  sinX100_24  cosX100_

#### Save dataframe to a csv file

In [9]:
df_new.to_csv(EXTDATA_DIR + 'test/GFS/' + 'testGFS.csv', index=False)